In [18]:
import h5py
import numpy as np
from PIL import Image
from skimage import io
from skimage.viewer import ImageViewer
import psycopg2

In [2]:
f = h5py.File('AnatomyLabelDatabase.hdf5', 'r')

#f2 = h5py.File('ZBBDatabase.hdf5', 'r')

In [4]:
for key in f.keys():
    print key

6.7FRhcrtR-Gal4-uasKaede_6dpf_MeanImageOf12Fish
Anti-5HT_6dpf_MeanImageOf40Fish
Anti-Gad67_6dpf_MeanImageOf17Fish
Anti-GlyR_6dpf_MeanImageOf160Fish
Anti-TH_6dpf_MeanImageOf10Fish
Anti-Zn12(Hnk-1)_6dpf_MeanImageOf9Fish
Anti-Zn1_6dpf_MeanImageOf10Fish
Anti-Znp1(Synaptotagmin2)_6dpf_MeanImageOf118Fish
Anti-Zrf1(GFAP)_6dpf_MeanImageOf5Fish
Anti-Zrf2_6dpf_MeanImageOf5Fish
Anti-tERK_6dpf_MeanImageOf193Fish
Elavl3-GCaMP5G_6dpf_MeanImageOf7Fish
Elavl3-H2BRFP_6dpf_MeanImageOf10Fish
EtVmat2-GFP_6dpf_MeanImageOf55Fish
Gad1b-GFP_6dpf_MeanImageOf10Fish
Glyt2-GFP_6dpf_MeanImageOf13Fish
Hcrt-RFP_6dpf_MeanImageOf15Fish
Isl1-GFP_6dpf_MeanImageOf17Fish
Isl2bGal4-uasDendra_6dpf_MeanImageOf8Fish
Olig2-GFP_6dpf_MeanImageOf12Fish
Otpb.A-Gal4-UAS-GCaMP_6-7dpf_MeanImageOf19Fish
Otpb.A-Nsfb-GFP_6dpf_MeanImageOf10Fish
Oxtl-GFP_6dpf_MeanImageOf68Fish
Pet1-GFP_6dpf_MeanImageOf13Fish
Ptf1aGal4-uasKaede_6dpf_MeanImageOf11Fish
Qrfp-GFP_6dpf_MeanImageOf15Fish
S1181tGal4-uasKaede_6dpf_MeanImageOf14Fish
SpinalBackfills

In [17]:
data = f.get('Elavl3-H2BRFP_6dpf_MeanImageOf10Fish')
#viewer = ImageViewer(data[100])
#viewer.show()

(138, 621, 1406)

In [10]:
io.imsave('Test/test_16bit_jpeg.jpg', data[86])

In [11]:
for key in f.keys()[:2]:
    stack = f.get(key)
    for i, img in enumerate(stack):
        fpath = 'Test/%s-%i.jpg' % (key, i)

In [19]:
conn = psycopg2.connect(dbname='zbrain2db', user='gqe', host='localhost', password='')
cursor = conn.cursor()